# Sprint ディープラーニングフレームワーク2

## 【問題3】Iris（2値分類）をKerasで学習
TensorFlowによるIrisデータセットに対する2値分類をKerasに書き換えてください。

In [5]:
import tensorflow as tf
from tensorflow import keras

from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from keras.datasets import mnist
from keras.utils import np_utils
from keras import backend as K
from sklearn.preprocessing import OneHotEncoder

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [6]:
print(tf.__version__)

1.13.1


In [7]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

In [8]:
"""
Kerasを使いIrisデータセットを2値分類する
"""
# データセットの読み込み
dataset_path ="../Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y = y.astype(np.int)[:, np.newaxis]
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [9]:
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
# n_hidden1 = 50
# n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

In [10]:
model = Sequential()
model.add(Dense(input_dim=n_input, units=n_classes)) # 入力が4種の特徴、2つに分離
model.add(Activation('softmax'))
model.compile(loss='binary_crossentropy', optimizer=SGD(lr=learning_rate))
model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size) # エポック10, ミニバッチ 10

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
64/64 [==============================] - 0s 5ms/step - loss: 7.4730
Epoch 2/10
64/64 [==============================] - 0s 155us/step - loss: 7.4730
Epoch 3/10
64/64 [==============================] - 0s 142us/step - loss: 7.4730
Epoch 4/10
64/64 [==============================] - 0s 147us/step - loss: 7.4730
Epoch 5/10
64/64 [==============================] - 0s 154us/step - loss: 7.4730
Epoch 6/10
64/64 [==============================] - 0s 152us/step - loss: 7.4730
Epoch 7/10
64/64 [==============================] - 0s 172us/step - loss: 7.4730
Epoch 8/10
64/64 [==============================] - 0s 160us/step - loss: 7.4730
Epoch 9/10
64/64 [==============================] - 0s 144us/step - loss: 7.4730
Epoch 10/10
64/64 [==============================] - 0s 151us/step - loss: 7.4730


### 実行できることまで確認

### 2周目として追加学習
- 中間層やaccuracy, valデータを追加

In [11]:
K.clear_session()

model = Sequential()
model.add(Dense(32, activation='relu', input_dim=n_input))  # 中間層
model.add(Dense(input_dim=n_input, units=n_classes)) # 0or1の2つに分離
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=SGD(lr=learning_rate), metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=num_epochs, batch_size=batch_size) # エポック10, ミニバッチ 10

Train on 64 samples, validate on 16 samples
Epoch 1/10
64/64 [==============================] - 0s 7ms/step - loss: 0.7265 - accuracy: 0.4375 - val_loss: 0.6917 - val_accuracy: 0.6875
Epoch 2/10
64/64 [==============================] - 0s 209us/step - loss: 0.6896 - accuracy: 0.5625 - val_loss: 0.7185 - val_accuracy: 0.3750
Epoch 3/10
64/64 [==============================] - 0s 198us/step - loss: 0.6779 - accuracy: 0.5312 - val_loss: 0.7166 - val_accuracy: 0.3750
Epoch 4/10
64/64 [==============================] - 0s 190us/step - loss: 0.6775 - accuracy: 0.5312 - val_loss: 0.7117 - val_accuracy: 0.3750
Epoch 5/10
64/64 [==============================] - 0s 196us/step - loss: 0.6704 - accuracy: 0.5625 - val_loss: 0.7149 - val_accuracy: 0.3750
Epoch 6/10
64/64 [==============================] - 0s 193us/step - loss: 0.6682 - accuracy: 0.5469 - val_loss: 0.6935 - val_accuracy: 0.3750
Epoch 7/10
64/64 [==============================] - 0s 192us/step - loss: 0.6400 - accuracy: 0.5312 - val_

## 【問題4】Iris（多値分類）をKerasで学習
TensorFlowによるIrisデータセットに対する3値分類をKerasに書き換えてください。

In [12]:
"""
Kerasを使いIrisデータセットを3値分類する
☆重要：他クラス分類はワンホットエンコーディングが必要、n_classesはワンホットした分だけ必要。

"""
from keras.utils import np_utils

# データセットの読み込み
dataset_path ="../Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
#########変更############
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
#########変更############
y[y=='Iris-setosa'] = 0
y[y=='Iris-versicolor'] = 1
y[y=='Iris-virginica'] = 2
y = y.astype(np.int)[:, np.newaxis]

# kerasメソッドによるOne-hotエンコーディング
y = np_utils.to_categorical(y) # 数値を、位置に変換 [0,1,2] ==> [ [1,0,0],[0,1,0],[0,0,1] ]

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [14]:
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 32
n_hidden2 = 16
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 3

model2 = Sequential()
model2.add(Dense(input_dim=n_input, units=n_classes))
model2.add(Activation('softmax'))
model2.compile(loss='categorical_crossentropy', optimizer=SGD(lr=learning_rate))
model2.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size) # エポック10, ミニバッチ 10
pred_model2 = model2.predict_classes(X_test, batch_size=10)

Epoch 1/10
96/96 [==============================] - 0s 3ms/step - loss: 2.4056
Epoch 2/10
96/96 [==============================] - 0s 169us/step - loss: 1.5910
Epoch 3/10
96/96 [==============================] - 0s 197us/step - loss: 1.0699
Epoch 4/10
96/96 [==============================] - 0s 155us/step - loss: 0.8244
Epoch 5/10
96/96 [==============================] - 0s 154us/step - loss: 0.7419
Epoch 6/10
96/96 [==============================] - 0s 142us/step - loss: 0.7275
Epoch 7/10
96/96 [==============================] - 0s 145us/step - loss: 0.7130
Epoch 8/10
96/96 [==============================] - 0s 150us/step - loss: 0.6866
Epoch 9/10
96/96 [==============================] - 0s 155us/step - loss: 0.6916
Epoch 10/10
96/96 [==============================] - 0s 150us/step - loss: 0.6637


### 実行できることまで確認

### 2周目として追加学習
- 中間層やaccuracy, valデータを追加

In [15]:
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 32
n_hidden2 = 16
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 3

K.clear_session()

model2 = Sequential()
model2.add(Dense(n_hidden1, activation='relu', input_dim=n_input))  # 中間層1
model2.add(Dense(n_hidden2, activation='relu'))  # 中間層2
model2.add(Dense(input_dim=n_hidden2, units=n_classes)) # 3値分類
model2.add(Activation('softmax'))
model2.compile(loss='categorical_crossentropy', optimizer=SGD(lr=learning_rate), metrics=['accuracy'])
model2.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=num_epochs, batch_size=batch_size) # エポック10, ミニバッチ 10

Train on 96 samples, validate on 24 samples
Epoch 1/10
96/96 [==============================] - 0s 4ms/step - loss: 1.1672 - accuracy: 0.3750 - val_loss: 1.0150 - val_accuracy: 0.3750
Epoch 2/10
96/96 [==============================] - 0s 217us/step - loss: 0.9940 - accuracy: 0.4271 - val_loss: 0.9480 - val_accuracy: 0.6250
Epoch 3/10
96/96 [==============================] - 0s 263us/step - loss: 0.9326 - accuracy: 0.5833 - val_loss: 0.8763 - val_accuracy: 0.7083
Epoch 4/10
96/96 [==============================] - 0s 246us/step - loss: 0.8710 - accuracy: 0.7604 - val_loss: 0.8466 - val_accuracy: 0.7083
Epoch 5/10
96/96 [==============================] - 0s 229us/step - loss: 0.8561 - accuracy: 0.6771 - val_loss: 0.8236 - val_accuracy: 0.7917
Epoch 6/10
96/96 [==============================] - 0s 243us/step - loss: 0.7877 - accuracy: 0.8333 - val_loss: 0.7571 - val_accuracy: 0.9167
Epoch 7/10
96/96 [==============================] - 0s 227us/step - loss: 0.7439 - accuracy: 0.7708 - val_

## 【問題5】House PricesをKerasで学習
TensorFlowによるHouse Pricesデータセットに対する回帰をKerasに書き換えてください。

In [16]:
"""
Kerasを使いHouse Priceの回帰問題
"""
# データセットの読み込み
dataset_path ="../train.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
# データフレームから条件抽出
#########変更############
y = df["SalePrice"]
X = df.loc[:, ["GrLivArea", "YearBuilt"]]
y = np.array(y)
X = np.array(X)
y = y.astype(np.int)[:, np.newaxis]

#正規化
ss = StandardScaler()
ss.fit(X)
X_scaler = ss.transform(X)

# yを対数変換
y_log = np.log(y)

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X_scaler, y_log, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

/Users/adachi-yuya/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/adachi-yuya/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [17]:
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 50
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

K.clear_session()

model3 = Sequential()
model3.add(Dense(input_dim=n_input, units=n_classes)) # 入力が4種の特徴、2つに分離
model3.add(Activation('relu'))
model3.compile(loss='mse', optimizer='rmsprop')
model3.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size) # エポック10, ミニバッチ 50

Epoch 1/10
934/934 [==============================] - 0s 193us/step - loss: 134.4240
Epoch 2/10
934/934 [==============================] - 0s 27us/step - loss: 133.9350
Epoch 3/10
934/934 [==============================] - 0s 26us/step - loss: 133.4952
Epoch 4/10
934/934 [==============================] - 0s 28us/step - loss: 133.0671
Epoch 5/10
934/934 [==============================] - 0s 21us/step - loss: 132.6358
Epoch 6/10
934/934 [==============================] - 0s 21us/step - loss: 132.1994
Epoch 7/10
934/934 [==============================] - 0s 21us/step - loss: 131.7686
Epoch 8/10
934/934 [==============================] - 0s 20us/step - loss: 131.3340
Epoch 9/10
934/934 [==============================] - 0s 22us/step - loss: 130.8974
Epoch 10/10
934/934 [==============================] - 0s 23us/step - loss: 130.4625


### 実行できることまで確認

### 2周目として追加学習
- 中間層とvalデータを追加

In [18]:
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 50
num_epochs = 10
n_hidden1 = 16
n_hidden2 = 8
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

K.clear_session()

model3 = Sequential()
model3.add(Dense(n_hidden1, activation='relu', input_dim=n_input))  # 中間層1
model3.add(Dense(n_hidden2, activation='relu'))  # 中間層2
model3.add(Dense(input_dim=n_hidden2, units=n_classes)) # 3値分類
model3.add(Activation('relu'))
model3.compile(loss='mse', optimizer="rmsprop")
model3.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=num_epochs, batch_size=batch_size) # エポック10, ミニバッチ 50

Train on 934 samples, validate on 234 samples
Epoch 1/10
934/934 [==============================] - 0s 440us/step - loss: 143.0841 - val_loss: 140.3077
Epoch 2/10
934/934 [==============================] - 0s 33us/step - loss: 139.0267 - val_loss: 135.9439
Epoch 3/10
934/934 [==============================] - 0s 31us/step - loss: 134.1354 - val_loss: 130.5497
Epoch 4/10
934/934 [==============================] - 0s 29us/step - loss: 128.1070 - val_loss: 124.0137
Epoch 5/10
934/934 [==============================] - 0s 30us/step - loss: 121.2765 - val_loss: 117.1270
Epoch 6/10
934/934 [==============================] - 0s 32us/step - loss: 114.2086 - val_loss: 109.9141
Epoch 7/10
934/934 [==============================] - 0s 32us/step - loss: 106.8339 - val_loss: 102.3623
Epoch 8/10
934/934 [==============================] - 0s 30us/step - loss: 99.3183 - val_loss: 94.8537
Epoch 9/10
934/934 [==============================] - 0s 28us/step - loss: 91.6193 - val_loss: 87.0496
Epoch 10/10


## 【問題6】MNISTをKerasで学習
TensorFlowによるMNISTデータセットによる画像の多値分類をKerasに書き換えてください。

In [19]:
# データセットの読み込みから前処理まで
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)

X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255

# kerasメソッドによるOne-hotエンコーディング
y_train_one_hot = np_utils.to_categorical(y_train) # 数値を、位置に変換 [0,1,2] ==> [ [1,0,0],[0,1,0],[0,0,1] ]
y_test_one_hot = np_utils.to_categorical(y_test) # 数値を、位置に変換 [0,1,2] ==> [ [1,0,0],[0,1,0],[0,0,1] ]

# さらにtrainとvalに分割
X_train, X_val, y_train_one_hot, y_val_one_hot = train_test_split(X_train, y_train_one_hot, test_size=0.2, random_state=0)
    

In [20]:
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 100
num_epochs = 20
n_hidden1 = 400
n_hidden2 = 200
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 10   # One-Hotエンコーディングした分

K.clear_session()

model4 = Sequential()
model4.add(Dense(input_dim=n_input, units=n_classes)) # 入力が4種の特徴、2つに分離
model4.add(Activation('softmax'))
model4.compile(loss='categorical_crossentropy', optimizer='rmsprop')
model4.fit(X_train, y_train_one_hot, epochs=num_epochs, batch_size=batch_size) # エポック20, ミニバッチ 100

Epoch 1/20
48000/48000 [==============================] - 2s 34us/step - loss: 0.6141
Epoch 2/20
48000/48000 [==============================] - 1s 24us/step - loss: 0.3386
Epoch 3/20
48000/48000 [==============================] - 1s 28us/step - loss: 0.3088
Epoch 4/20
48000/48000 [==============================] - 1s 28us/step - loss: 0.2943
Epoch 5/20
48000/48000 [==============================] - 1s 24us/step - loss: 0.2860
Epoch 6/20
48000/48000 [==============================] - 1s 29us/step - loss: 0.2799
Epoch 7/20
48000/48000 [==============================] - 1s 30us/step - loss: 0.2753
Epoch 8/20
48000/48000 [==============================] - 1s 28us/step - loss: 0.2718
Epoch 9/20
48000/48000 [==============================] - 2s 31us/step - loss: 0.2688
Epoch 10/20
48000/48000 [==============================] - 1s 31us/step - loss: 0.2666
Epoch 11/20
48000/48000 [==============================] - 2s 32us/step - loss: 0.2647
Epoch 12/20
48000/48000 [===========================

### 実行できることまで確認

### 2周目として追加学習
- 中間層とacc, valデータを追加

In [21]:
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 100
num_epochs = 20
n_hidden1 = 400
n_hidden2 = 200
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 10   # One-Hotエンコーディングした分

K.clear_session()

model4 = Sequential()
model4.add(Dense(n_hidden1, activation='relu', input_dim=n_input))  # 中間層1
model4.add(Dense(n_hidden2, activation='relu'))  # 中間層2
model4.add(Dense(input_dim=n_hidden2, units=n_classes)) # 3値分類
model4.add(Activation('softmax'))
model4.compile(loss='categorical_crossentropy', optimizer="rmsprop", metrics=['accuracy'])
model4.fit(X_train, y_train_one_hot, validation_data=(X_val, y_val_one_hot), epochs=num_epochs, batch_size=batch_size) # エポック20, ミニバッチ 100

Train on 48000 samples, validate on 12000 samples
Epoch 1/20
48000/48000 [==============================] - 6s 120us/step - loss: 0.2477 - accuracy: 0.9239 - val_loss: 0.1256 - val_accuracy: 0.9605
Epoch 2/20
48000/48000 [==============================] - 5s 114us/step - loss: 0.0951 - accuracy: 0.9703 - val_loss: 0.0908 - val_accuracy: 0.9713
Epoch 3/20
48000/48000 [==============================] - 6s 116us/step - loss: 0.0632 - accuracy: 0.9809 - val_loss: 0.1002 - val_accuracy: 0.9718
Epoch 4/20
48000/48000 [==============================] - 5s 111us/step - loss: 0.0447 - accuracy: 0.9859 - val_loss: 0.0751 - val_accuracy: 0.9793
Epoch 5/20
48000/48000 [==============================] - 6s 117us/step - loss: 0.0328 - accuracy: 0.9899 - val_loss: 0.0932 - val_accuracy: 0.9748
Epoch 6/20
48000/48000 [==============================] - 5s 115us/step - loss: 0.0258 - accuracy: 0.9918 - val_loss: 0.0913 - val_accuracy: 0.9763
Epoch 7/20
48000/48000 [==============================] - 6s 1

## 【問題7】（アドバンス課題）PyTorchへの書き換え
4種類の問題をPyTorchに書き換えてください。

## 【問題8】（アドバンス課題）フレームワークの比較
それぞれのフレームワークにはどのような違いがあるかをまとめてください。


《視点例》


- 計算速度
- コードの行数・可読性
- 用意されている機能

### 簡単に調査した内容
- keras
  - 所感) 現段階では個人的にkerasの方が使いやすい印象を持っている。Pytorchの調査量不足もあるが。
  - メリット) 書きやすい。compileやfitなどのメソッドにおいて引数指定で簡単に情報が取得できる点が良い。
  
- pytorch
  - 所感) fitメソッドがないのがsklearnやkerasと比べ劣る。クラスを作らないと走らないのは難しい印象。
  - メリット) numpyに似たメソッドがあるのは良い。実務的に想像するに、どのフレームワークを使っていても、デバック（テスト）環境を作りたい時があるはずで、pytorchはその点が楽。numpyをインストールしていない環境等でもpytorchで一気通貫で実行できそう。
  - デメリット) fitまでのlayer・activation・optimizerなどの構築が大変。